In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from html_to_markdown import convert_to_markdown
from IPython.display import display, Markdown

In [ ]:
from scifeeder.utils import read_papers_csv
from scifeeder.types import Paper
papers=list(read_papers_csv('suba_j.csv'))

In [ ]:
from collections import Counter
counts = Counter()
for p in papers:
    if p.issn and p.doi:
        counts[p.issn]+=1

In [ ]:
from scifeeder.selenium_cls import Selenium
from scifeeder.issn import ISSN_MAP

In [ ]:
paper = None
for idx, p1 in enumerate(reversed(papers)):
    if p1.issn:
        if p1.issn in {"0253-9772"}:
            continue
    if (p1.issn not in DATA) and p1.doi and p1.year > 1980 and counts[p1.issn] > 0:
        paper = p1
        break
print(idx, len(papers), len(counts), len(ISSN_MAP))
if paper:
    paper.issn in ISSN_MAP

In [ ]:
paper = papers[311]

In [ ]:
f"https://doi.org/{paper.doi}"

In [ ]:
paper

In [ ]:
def toline(p):
    return f'"{p.issn}": {p.key},'
print(toline(paper))

In [ ]:
from scifeeder.selenium_cls import Selenium, Soup, StealthSelenium

from scifeeder.issn import ISSN_MAP

assert paper.issn in ISSN_MAP and paper.doi, paper
# web = StealthSelenium(timeout=15, headless=True)
web = Selenium(timeout=15, headless=False)
markdown, soup = web.run(paper.doi, ISSN_MAP[paper.issn])
if markdown:
    with open('saved.md', 'wt') as fp:
        fp.write(markdown)
Markdown(markdown)

In [ ]:
assert paper.issn in DATA and paper.doi, paper
markdown, soup = web.run(paper.doi, DATA[paper.issn])
if markdown:
    with open('saved.md', 'wt') as fp:
        fp.write(markdown)
Markdown(markdown)

In [ ]:
HTML(web.find_html())

----

## EPMC

In [ ]:
from bs4 import BeautifulSoup
from io import StringIO, BytesIO
from html_to_markdown import convert_to_markdown
from IPython.display import HTML, Markdown

In [ ]:
from scifeeder.epmc import EPMC
from scifeeder.xml_utils import Events, PMCEvents, PMCTAGS

In [ ]:
from IPython.display import HTML

In [ ]:
DONE= {'PMC5486124', 'PMC5447075', 'PMC5853234', 'PMC5490910', 'PMC5490927', 'PMC5823009', 'PMC5399074',
      'PMC5575511', 'PMC5387099', 'PMC5405513', 'PMC5412409', 'PMC5462045', 'PMC5444443', 'PMC5435427'}

In [ ]:

for p in reversed(papers):
    if p.pmcid and p.pmcid not in DONE and p.year > 2010:
        break
p

In [ ]:
f"https://doi.org/{p.doi}"

In [ ]:
CSS="""
<style>
.email { color:orange; }
.title-group { color: blue; font-size: 14pt; }
figure { border: solid orange 1px; padding: 0.5em; }
.missing { font-size:larger; color: red; }
.sc { color: red; font-weight: bold; }
</style>
"""

In [ ]:
x =  EPMC.from_pmcid(p.pmcid)
html = x.html()
if x.missing:
    print(x.missing)
HTML(CSS + html)

In [ ]:
# x.save_content('saved.xml')

In [ ]:
x.save_content("saved.xml", pretty=True)

----

## Runner

In [ ]:
from scifeeder.runner import Runner
from scifeeder.cache import Cache
from scifeeder.epmc import EPMC
from scifeeder.ncbi import NCBI

In [ ]:
from scifeeder.utils import read_papers_csv
from scifeeder.types import Paper
papers=list(read_papers_csv('suba_j.csv'))

In [ ]:
DONE = {'11511543', '20847098', '21051552'}

In [ ]:
for p in papers:
    if p.pmcid and p.pmid not in DONE and p.year > 2010:
        break
p

In [ ]:
from scifeeder.ncbi import NCBI

In [ ]:
x = NCBI.from_pmcid(p.pmcid)

In [ ]:
markdown = x.article()
Markdown(markdown)

In [ ]:
len([p for p in papers if p.pmcid]), len(papers), len([p for p in papers if p.doi])

In [ ]:
what = [p for p in papers if not p.doi and not p.pmcid]

for p in what:
    url=f"https://ncbi.nlm.nih.gov/pubmed/{p.pmid}/"
    print(p.pmid, p.year, url)

In [ ]:
html = x.html(True)
with open('saved.html', 'wt') as fp:
    fp.write(html)

In [ ]:
cache = Cache('mycache')
cache.fetch_ncbi(p)

In [ ]:
from scifeeder.runner import Runner
from scifeeder.selenium_cls import Selenium, StealthSelenium
from scifeeder.cache import Cache

from scifeeder.issn import ISSN_MAP
class SeleniumRunner(Runner):

    def start(self):

        self.cache = Cache('scache')
        self.web = self.create_driver()
        self.ok = 0
        self.failed = 0

    def create_driver(self):
        return StealthSelenium(headless=True)
        
    def work(self, paper, tqdm):
        tqdm.write(f'working... {paper.pmid}')
        if not paper.doi:
            return 'nodoi'
        if paper.issn not in ISSN_MAP:
            return 'noissn'

        try:
            html =  self.web.fetch_html(paper.doi,ISSN_MAP[paper.issn])
            if html is None:
                self.web = self.create_driver()
                tqdm.write('retry....')
                html =  self.web.fetch_html(paper.doi,ISSN_MAP[paper.issn])
            if html is None:
                retval= 'cc'
            elif not html:
                retval= 'timeout'
            else:
                self.cache.save_html(paper, html)
                retval = 'ok'
        except Exception as e:
            tqdm.write(f'failed: {paper.pmid} {e}')
            retval= 'failed'
        tqdm.write(retval)
        return retval

    def end(self):
        self.web.close()
    


In [ ]:
runner = SeleniumRunner('suba_j.csv')

In [ ]:
runner.run(True)